# Wprowadzenie do konwolucyjnych sieci neuronowych

Za chwilę zagłębimy się w zagadnienia teoretyczne zwiazane z konwolucyjnymi sieciami neuronowymi. Dlaczego sieci te tak dobrze sprawdzają się podczas przetwarzania obrazu? Zacznijmy od przyjrzenia się prostemu przykładowi sieci konwolucyjnej. Sieć ta służy do klasyfikacji zbioru cyfr MNIST. Problemem tym zajmowaliśmy się już wcześniej. Mimo że sieć konwolucyjna, którą przedstawię za chwilę, będzie bardzo prosta, to uzyska ona o wiele lepszy wynik od modelu użytego wcześniej.

Poniższy kod przedstawia wygląd podstawowej sieci konwolucyjnej. Jest ona stosem warstw Conv2D i MaxPooling2D. Za chwilę opiszę szczegółowo działanie tych warstw.

Sieć neuronowa przyjmuje tensor o kształcie określanym przez wysokość obrazu, jego szerokość i kanały obrazu — nie wliczamy tutaj wymiaru próbki. W związku z tym sieć na przetwarzać obiekty wejściowe o rozmiarze (28, 28, 1) — taki jest właśnie format obrazów wchodzących w skład zbioru MNIST. W związku z tym do pierwszej warstwy sieci przekazujemy argument input_shape=(28, 28, 1).

In [1]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

2025-05-11 23:49:08.079560: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-11 23:49:08.101342: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747000148.120046  115676 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747000148.126360  115676 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-11 23:49:08.151145: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

Wyświetlmy aktualną architekturę sieci:

In [2]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 11, 11, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 3, 3, 64)       │        36,928 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 55,744 (217.75 KB)

 Trainable params: 55,744 (217.75 KB)

 Non-trainable params: 0 (0.00 B)

Na wyjściu każdej warstwy Conv2D i MaxPooling2D pojawia się trójwymiarowy tensor o kształcie (wysokość, szerokość, kanały). Wraz z zagłębianiem się asieci wysokość i szerokość mają tendencję do przyjmowania mniejszych wartości. Liczbę kanałów określa pierwszy argument przekazany do warstw Conv2D (możemy mieć np. 32 lub 64 kanały).

Kolejnym krokiem jest przekazanie ostatniego tensora wyjściowego o kształcie (3, 3, 64) do klasyfikatora w postaci sieci gęstej (ten typ sieci opisywałem wcześniej) — stosu warstw Dense. Klasyfikatory te przetwarzają jednowymiarowe wektory, a nasze dane mają postać trójwymiarowego tensora. W związku z tym musimy spłaszczyć nasze dane wyjściowe, a następnie dodać kilka górnych warstw Dense:


In [3]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

Klasyfikacja ma na celu dokonanie przyporządkowania do jednego z dziesięciu zbiorów, a więc będziemy korzystać z ostatniej warstwy generującej 10 wartości wyjściowych i użyjemy funkcji aktywacji softmax. Oto aktualna charakterystyka architektury sieci:

In [4]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 11, 11, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 3, 3, 64)       │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 576)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 93,322 (364.54 KB)

 Trainable params: 93,322 (364.54 KB)

 Non-trainable params: 0 (0.00 B)

Jak widać, tensory przed skierowaniem do dwóch warstw Dense, mając początkowo kształt (3, 3, 64) zostały spłaszczone do wektorów o kształcie (576, ).

Spróbujmy wytrenować sieć konwolucyjną na zbiorze MNIST.



In [5]:
from keras.datasets import mnist
from keras.utils import to_categorical

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [6]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=5, batch_size=64)

Epoch 1/5


I0000 00:00:1747000160.152839  115751 service.cc:148] XLA service 0x7f9154004bf0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1747000160.152881  115751 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 4070 Ti SUPER, Compute Capability 8.9
2025-05-11 23:49:20.176029: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1747000160.252307  115751 cuda_dnn.cc:529] Loaded cuDNN version 90900


 65/938 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5155 - loss: 1.4550

I0000 00:00:1747000162.204006  115751 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


938/938 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.8747 - loss: 0.3944
Epoch 2/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9833 - loss: 0.0547
Epoch 3/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9900 - loss: 0.0320
Epoch 4/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9920 - loss: 0.0250
Epoch 5/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9941 - loss: 0.0196


Sprawdźmy działanie modelu na testowym zbiorze danych:

In [7]:
test_loss, test_acc = model.evaluate(test_images, test_labels)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9881 - loss: 0.0414


In [8]:
test_acc

0.9907000064849854

Gęsta sieć zastosowana w celu rozwiązania tego problemu wcześniej. charakteryzowała się testową dokładnością na poziomie 97,8%, a podstawowa sieć konwolucyjna uzyskała wynik 99,3%. Względna wartość błędu została zmniejszona o 68%. Nieźle!